# Scraping from film-grab for Image Mode
For the "image" mode, we download the images on the site film-grab.com

In [ ]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from datetime import datetime
import zipfile

import pandas as pd
import pymysql


import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
FOLDER_SC = "/home/tanguy/data/movizz/screenshot"

### Download images

#### Get list of movie in film-grab

In [ ]:
url = 'https://film-grab.com/movies-a-z/'
req = requests.get(url)
soup = BeautifulSoup(req.text, "lxml")

l = [(s.text, s.a['href']) for s in soup.find('ul', {'class': 'display-posts-listing'}).find_all('li')]
df_movies = pd.DataFrame(l, columns=['name', 'url'])

#### Get tt id from imdb from movie name

In [ ]:
def get_tt_from_name(name):
    formatted_movie_title = urllib.parse.quote(name)
    url = f"https://www.imdb.com/find?q={formatted_movie_title}&s=tt&ttype=ft&ref_=fn_ft"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    tmp_id = soup.find_all("td", {"class": "result_text"})
    if len(tmp_id) != 0:
        movie_id = tmp_id[0].find('a')['href'][7:-1]
    else:
        movie_id = -1
    return movie_id

def get_tt_from_name_url(name, url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    str_url = soup.find_all('a', href=re.compile('https?://film-grab\.com/category/[1-3][0-9]{3}/'))[0]['href']
    year = re.search('https?://film-grab\.com/category/([1-3][0-9]{3})/', str_url).group(1)
    
    formatted_movie_title = urllib.parse.quote(name + ' ' + year)
    url2 = f"https://www.imdb.com/find?q={formatted_movie_title}&s=tt&ttype=ft&ref_=fn_ft"
    req2 = requests.get(url2)
    soup2 = BeautifulSoup(req2.text, "lxml")
    tmp_id = soup2.find_all("td", {"class": "result_text"})
    if len(tmp_id) != 0:
        movie_id = tmp_id[0].find('a')['href'][7:-1]
    else:
        movie_id = -1
    return movie_id

In [ ]:
list_tt = []
for index, row in df_movies.iterrows():
    name = row['name']
    url = row['url']
#     tt = get_tt_from_name_url(name, url)
    tt = get_tt_from_name(name)
    print(tt)
    list_tt.append(tt)
    
df_movies['tt'] = list_tt

#### Download files

In [ ]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
list_dir = [d for d in [dirs for root, dirs, files in os.walk(FOLDER_SC)]][0]

for index, row in df_movies.iterrows():
    name = row['name']
    url = row['url']
    tt = row['tt']
    print(name, tt, end=" ")
    if tt in list_dir:
        print('(already dl)')
        continue
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    url_zip_tmp = soup.find('div', {'class': 'bwg_download_gallery'})
    if url_zip_tmp == None:
        print('(no button)')
        continue
    else:
        url_zip = url_zip_tmp.a['href']
    
    tmp_path = f"/home/tanguy/data/movizz/zip/tmp_{tt}.zip"
    path = os.path.join(FOLDER_SC, tt)
    
    download_url(url_zip, tmp_path)
    
    with zipfile.ZipFile(tmp_path,"r") as zip_ref:
        zip_ref.extractall(path)
    
    print()

#### Rename tt 

In [ ]:
list_dir = [d for d in [dirs for root, dirs, files in os.walk(FOLDER_SC)]][0]

for tt in list_dir:
    print(tt)
    list_f_tt = [f for f in [files for root, dirs, files in os.walk(f"/home/tanguy/data/movizz/screenshot/{tt}/")]][0]
    for i, f_tt in enumerate(list_f_tt):
        ext = f_tt.split('.')[-1]
        source_path = f"/home/tanguy/data/movizz/screenshot/{tt}/{f_tt}"
        target_path = f"/home/tanguy/data/movizz/screenshot/{tt}/{i}.{ext}"
        os.rename(source_path, target_path)

### Fill database

In [ ]:
# Connect to the database
connection = pymysql.connect(host='localhost',user='root',password='',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

list_dir = [d for d in [dirs for root, dirs, files in os.walk(FOLDER_SC)]][0]

for tt in list_dir:
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT id from quizz_movie WHERE imdb_id = '{tt}'")
        res = cursor.fetchall()
    connection.commit()
    print(tt, end='')
    
    if len(res) == 0:
        print('(no movie in database)')
        continue
    else:
        id_movie_db = res[0]['id']
        
        
    
    list_f_tt = [f for f in [files for root, dirs, files in os.walk(f"/home/tanguy/workspace/git/guess_movie/data_process/screenshot/{tt}/")]][0]
    
    for f_tt in list_f_tt:
        str_image = f"{tt}/{f_tt}"
        print(str_image)    
    
        with connection.cursor() as cursor:
            # Create a new record
            try:
                sql = "INSERT INTO `quizz_screenshot` (`movie_id`, `image`) VALUES (%s, %s)"
                cursor.execute(sql, (id_movie_db, str_image))
            except:
                print('E',end='')

    connection.commit()
#     break
    
    
connection.close()

#### Update has_image field

In [ ]:
connection = pymysql.connect(host='localhost',user='root',password='',db='quizz_db',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

list_dir = [d for d in [dirs for root, dirs, files in os.walk(FOLDER_SC)]][0]

for tt in list_dir:
    print(tt, end='')
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT id from quizz_movie WHERE imdb_id = '{tt}'")
        res = cursor.fetchall()
    connection.commit()
    print(tt, end='')
    
    if len(res) == 0:
        print('(no movie in database)')
        continue
    else:
        id_movie_db = res[0]['id']
        
    ##########################################    
    # Add has_quote field
    ##########################################
    
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT count(*) from quizz_screenshot WHERE movie_id = '{id_movie_db}'")
        res = cursor.fetchall()
    connection.commit()
    
    if res[0]['count(*)'] != 0:
        with connection.cursor() as cursor:
            cursor.execute(f"UPDATE quizz_movie SET has_image=1 WHERE id='{id_movie_db}'")
        connection.commit()
        
        print('[OK]')
    
connection.close()